In [4]:
import pandas as pd
from config.fucntions import columns_converter as cc
import os
import re
import numpy as np

# MVP
Dataset with pictures from given dataset  
- added from internet
- links
- a new excel table with names of each file, link to the object, and label (each label = the ID for one caption).

There is a file with each unique caption associated with a number. (Because one label in original file associates with many files)

In [5]:
file_name = 'datasets/production/dataset_CLEAN.csv'
df_clean = pd.read_csv(file_name, sep=';', encoding='utf-8')
df_clean = df_clean.drop(columns=['archive_links', 'microfiche_archive_links'])
cc.strings_to_links(df_clean, df_clean.columns[3:])

file_name = 'config/variables/blueprint_prefixes.txt'
with open(file_name, 'r') as f:
    blueprint_prefixes  = f.read().splitlines()

In [6]:
# Takes 5 min
df_mvp = pd.DataFrame({'id': [], 'id_object': [], 
                       'label': [], 'file_name': [], 'link': []})

for index, obj in df_clean.iterrows():

    if len(obj.image_links):
        file_type, iter_column = 'img', obj.image_links
    elif len(obj.microfiche_links):
        file_type, iter_column = 'scn', obj.microfiche_links
    else:
        continue

    
    for i, link in enumerate(iter_column):

        if any(prefix in link.split('/')[-1] for prefix in blueprint_prefixes):
            continue
            # file_type = 'blp'

        name = ''
        name += re.findall(r'.*/([^/]*)$', link)[0]
        name = file_type + '-' + name

        if not df_mvp[(df_mvp.id == link.split('/')[-1]) & (df_mvp.id_object == str(index))].shape[0]:
            df_mvp = df_mvp._append({'id': link.split('/')[-1], 
                                    'id_object': str(index),
                                    'label': obj.label,
                                    'file_name': name,
                                    'link': link}, ignore_index=True)


KeyboardInterrupt: 

Here:
- `id` - unique id of picture
- `id_object` - id (loc) of an object (row) in `df_CLEAN.csv`, to which this picture below
- `label` - label for this image in this object
- `file_name` - related filename in folder for this picture
- `link` - link to download 

In [8]:
print('All objects are uniwue (id): ', df_mvp.id.unique().shape[0] == df_mvp.shape[0])
df_mvp[df_mvp.file_name == 'img-fmd463147']

All objects are uniwue (id):  False


,id,id_object,label,file_name,link
12399,fmd463147,8447,Landgrafenschloss Landgrafenschloss/Leutehaus,img-fmd463147,http://www.bildindex.de/bilder/d/fmd463147
12452,fmd463147,8448,Landgrafenschloss Landgrafenschloss/Leutehaus ...,img-fmd463147,http://www.bildindex.de/bilder/d/fmd463147
12497,fmd463147,8451,Landgrafenschloss Landgrafenschloss/Leutehaus ...,img-fmd463147,http://www.bildindex.de/bilder/d/fmd463147
12503,fmd463147,8452,Landgrafenschloss Landgrafenschloss/Leutehaus ...,img-fmd463147,http://www.bildindex.de/bilder/d/fmd463147
12523,fmd463147,8454,Landgrafenschloss Landgrafenschloss/Leutehaus ...,img-fmd463147,http://www.bildindex.de/bilder/d/fmd463147
12526,fmd463147,8455,Landgrafenschloss Landgrafenschloss/Leutehaus ...,img-fmd463147,http://www.bildindex.de/bilder/d/fmd463147
12565,fmd463147,8456,Landgrafenschloss Landgrafenschloss/Leutehaus ...,img-fmd463147,http://www.bildindex.de/bilder/d/fmd463147
12610,fmd463147,8459,Landgrafenschloss Landgrafenschloss/Leutehaus ...,img-fmd463147,http://www.bildindex.de/bilder/d/fmd463147
12616,fmd463147,8460,Landgrafenschloss Landgrafenschloss/Leutehaus ...,img-fmd463147,http://www.bildindex.de/bilder/d/fmd463147
12636,fmd463147,8462,Landgrafenschloss Landgrafenschloss/Leutehaus ...,img-fmd463147,http://www.bildindex.de/bilder/d/fmd463147


Turned out that there are repetitions of pictures among different objects. On this step dataset above could be saved, but we will keep working with that, which consists only from unique links.

In [ ]:
# for ind, obj in df_mvp.iterrows():
#     if obj.link == df_mvp['link'].value_counts()[0]:
#         print(obj.id, '\t', obj.file_name,'\t', obj.label)

In [9]:
df_mvp = df_mvp.drop(columns=['id']).groupby('link').agg(list).reset_index()
df_mvp['id'] = df_mvp['link'].apply(lambda x: x.split('/')[-1])
df_mvp['file_name'] = df_mvp['file_name'].apply(lambda x: x[0])
df_mvp.rename(columns={'label': 'labels'}, inplace=True)
df_mvp.rename(columns={'id_object': 'id_objects'}, inplace=True)

print('All links are unique:', df_mvp.link.unique().shape[0] == df_mvp.shape[0])
print('All ids are unique:', df_mvp.id.unique().shape[0] == df_mvp.shape[0])
print('All file names are unique:', df_mvp.file_name.unique().shape[0] == df_mvp.shape[0])
print()
print('Amount of pictures:', df_mvp.shape[0])

df_mvp.head()

All links are unique: True
All ids are unique: True
All file names are unique: True

Amount of pictures: 31922


,link,id_objects,labels,file_name,id
0,http://www.bildindex.de/bilder/d/ae00001b05,[9077],[Negativ (Fotografie)],scn-ae00001b05,ae00001b05
1,http://www.bildindex.de/bilder/d/ae00001b06,[9077],[Negativ (Fotografie)],scn-ae00001b06,ae00001b06
2,http://www.bildindex.de/bilder/d/ae00001b07,[9077],[Negativ (Fotografie)],scn-ae00001b07,ae00001b07
3,http://www.bildindex.de/bilder/d/ae00001b08,[9077],[Negativ (Fotografie)],scn-ae00001b08,ae00001b08
4,http://www.bildindex.de/bilder/d/ae00001b10,[3899],[Einheimische bei der Arbeit],scn-ae00001b10,ae00001b10


In [11]:
dict_labels = dict(map(lambda label, id_label : 
                       (label, id_label), 
                       df_clean['label'].unique(), 
                       np.array(range(len(df_clean['label'].unique())))))

df_mvp['labels'] = df_mvp['labels'].apply(lambda labels: [dict_labels[label] for label in labels])
df_mvp.head()

,link,id_objects,labels,file_name,id
0,http://www.bildindex.de/bilder/d/ae00001b05,[9077],[1617],scn-ae00001b05,ae00001b05
1,http://www.bildindex.de/bilder/d/ae00001b06,[9077],[1617],scn-ae00001b06,ae00001b06
2,http://www.bildindex.de/bilder/d/ae00001b07,[9077],[1617],scn-ae00001b07,ae00001b07
3,http://www.bildindex.de/bilder/d/ae00001b08,[9077],[1617],scn-ae00001b08,ae00001b08
4,http://www.bildindex.de/bilder/d/ae00001b10,[3899],[3257],scn-ae00001b10,ae00001b10


`df_mvp` design:
- `link` - link to download 
- `id_objects` - indexes (loc) of objects (row) in `df_CLEAN.csv`, to which this picture below
- `labels` - labels for this image in these object
- `file_name` - related filename in folder for this picture
- `id` - unique id of picture

`df_labels` design:
- `id` - unique id of a label
- `label` - label (str)

Create a dataset with all unique labels and their ids 

In [338]:
# Creating dcit with all unique labels and there ids
df_labels = pd.DataFrame({'id': np.array(range(len(df_clean['label'].unique()))),
                          'label': df_clean['label'].unique()})
df_labels.head()

,id,label
0,0,"Lauenhain (Kreis Hainichen), Fahnenträger eine..."
1,1,Fenster (Bauelement)
2,2,"Kassel, Infanteriekasernen an der Königsstraße..."
3,3,"Waldeck, Domäne, Wohnhaus, Umbauentwurf, Aufrisse"
4,4,"Kassel, Opernhaus (Altes Hoftheater), Aborte i..."


Save dataset

In [ ]:
#### df_mvp.to_csv('datasets/production/dataset_MVP.csv', sep=';', encoding='utf-8', index=False)
#### df_labels.to_csv('datasets/production/dataset_labels.csv', sep=';', encoding='utf-8', index=False)